In [1]:
import multiprocessing, torch, re
import os.path as osp
import datasets

from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig,\
    DataCollatorForLanguageModeling, Trainer, TrainingArguments, BertForMaskedLM,\
    FillMaskPipeline, EncoderDecoderModel, BertTokenizerFast,\
    Seq2SeqTrainer, Seq2SeqTrainingArguments
from tokenization_kobert import KoBertTokenizer
from datasets import load_dataset, Dataset
from evaluate import load as load_metric
from tqdm.auto import tqdm
from time import strftime, time, localtime
from os import listdir
from functools import partial
from typing import List
from zipfile import ZipFile

print(torch.__version__)

2.0.0+cu117


In [2]:
# model_path = "skt/kogpt2-base-v2"
# tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path, 
#                                                     bos_token='</s>', eos_token='</s>', unk_token='<unk>',
#                                                     pad_token='<pad>', mask_token='<mask>', max_length=512)
# model = GPT2LMHeadModel.from_pretrained(model_path)
# model = BertForMaskedLM.from_pretrained("skt/kobert-base-v1")
# model = AutoModel.from_pretrained('monologg/distilkobert')
# model = BertForMaskedLM.from_pretrained("monologg/kobert-lm")
# tokenizer = KoBertTokenizer.from_pretrained('monologg/distilkobert')
model = EncoderDecoderModel.from_pretrained("kykim/bertshared-kor-base")
tokenizer = BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [3]:
model

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [4]:
# sent = "안녕하세요.[MASK][MASK][MASK][MASK][MASK][MASK]"

# collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
# encoded = tokenizer(sent, return_tensors="pt")
# target = tokenizer("안녕하세요", return_tensors='pt')

# out = model(**encoded)
# pipe = FillMaskPipeline(model=model, tokenizer=tokenizer)
# while True:
#     t = model(**encoded)
#     logits:torch.Tensor = t.logits
#     predicted_token = logits[:,-1,:].argmax(dim=1)
#     print(tokenizer.decode(predicted_token))

# print(tokenizer.decode(logits[:,-1,:].argmax(dim=1)))

In [5]:
# pipe("안녕하[MASK][MASK]")

In [6]:
# mask_idxs = encoded['input_ids'] == tokenizer.mask_token_id
# print(mask_idxs)
# print(out.logits.shape)
# masks = out.logits[mask_idxs].argmax(dim=-1)
# print(masks)
# print(tokenizer.decode(masks))
# predicted_token = out.logits[0,-1].argmax(-1)
# tokenizer.decode(predicted_token)

In [7]:
# dataset = load_dataset("HuggingFaceH4/ultrachat_200k")
# dataset = load_dataset("wikipedia", "20220301.en")
# dataset = dataset.remove_columns([col for col in dataset.column_names if col != 'text'])
# dataset = load_dataset("bookcorpus", split="train", streaming=True).with_format('torch')
# dataset = load_dataset("bookcorpus", split="train")
# len(dataset)

In [8]:
# def batch_iterator(batch_size=10000):
#     for i in tqdm(range(0, len(dataset['train']), batch_size)):
#         yield dataset['train'][i:i+batch_size]['text']
# if [_dir for _dir in listdir() if "tokenizer" in _dir] != []:
#     latest_tokenizer_path = sorted([_dir for _dir in listdir() if "tokenizer" in _dir])[-1]
#     tokenizer = AutoTokenizer.from_pretrained(latest_tokenizer_path)
# else:
#     # tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2").train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
#     tokenizer = AutoTokenizer.from_pretrained("skplanet/dialog-koelectra-small-discriminator").train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
#     tokenizer.save_pretrained(get_time_dir())

# tokenizer.pad_token = '<pad>'
# tokenizer.eos_token = '</s>'
# tokenizer.bos_token = '</s>'
# tokenizer.unk_token = '<unk>'
# tokenizer.mask_token = '<mask>'
# tokenizer.model_max_length = 32

In [9]:
def get_time_dir(): return f"tokenizer_{strftime('%m-%d-%H-%M', localtime(time()))}"

In [10]:
num_proc = multiprocessing.cpu_count()
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

The max length for the tokenizer is: 512


In [11]:
s = ["안녕하세요", "저는 트위치에서"]
print(tokenizer.batch_encode_plus(s))
tokenizer.convert_tokens_to_ids(tokenizer.batch_encode_plus(s))

{'input_ids': [[2, 16453, 3], [2, 14156, 20136, 17235, 8044, 3]], 'token_type_ids': [[0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1, 1, 1, 1]]}


[1, 1, 1]

In [39]:
def process_data_to_model_inputs(batch:datasets.arrow_dataset.Batch, tokenizer=tokenizer):
    import re
    
    for i, s in enumerate(batch['text']):
        for match in reversed([*re.finditer("^[0-9]* : ", s)]):
            s = s[:match.start()] + s[match.end():]
        batch['text'][i] =s
    
    label:List[str] = batch['text']
    assert type(label) == list
    ####################################################
    # label:List[str] operation here.
    ####################################################
    tokenized_inputs = tokenizer(
        batch['text'], max_length=tokenizer.model_max_length, padding="max_length", truncation=True,
    )
    tokenized_outputs = tokenizer(
        label, max_length=tokenizer.model_max_length, padding="max_length", truncation=True,
    )
    
    batch['input_ids'] = tokenized_inputs.input_ids
    batch['attention_mask'] = tokenized_inputs.attention_mask
    batch['decoder_input_ids'] = tokenized_outputs.input_ids
    batch['decoder_attention_mask'] = tokenized_outputs.attention_mask
    batch['labels'] = tokenized_outputs.input_ids.copy()
    
    batch['labels'] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]
    
    return batch

In [40]:
# mapped = dataset.map(partial(process_date_to_model_inputs, tokenizer=tokenizer), batched=True, remove_columns=['text'], num_proc=num_proc) # bug on windows
if osp.exists("data"):
    mapped = datasets.load_from_disk('data')
else:
    with ZipFile("data.zip", mode='r') as zf:
        dataset = Dataset.from_dict(
            {"text":[zf.read(fname).decode('UTF-8') for fname in zf.filelist]}
    )
    dataset = dataset.train_test_split(0.3)
    mapped = dataset.map(partial(process_data_to_model_inputs, tokenizer=tokenizer), batched=True, remove_columns=['text'])
    mapped.set_format("torch")
    mapped.save_to_disk("cache")
mapped

  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
        num_rows: 69056
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
        num_rows: 29596
    })
})

In [41]:
one = mapped['train'][0]

In [42]:
len(one['input_ids'])

512

In [43]:
len(one['decoder_input_ids'])

512

In [44]:
# mapped_sample['train'][0]

In [45]:
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

model.config.max_length = 512
model.config.early_stopping = True
# model.config.no_repeat_ngram_size = 1
model.config.length_penalty = 2.
model.config.repetition_penalty = 3.
model.config.num_beams = 10
model.config.vocab_size = model.config.encoder.vocab_size

In [46]:
rouge = load_metric("rouge")
def compute_metrics(pred):
    print(type(pred))
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [47]:
args = Seq2SeqTrainingArguments(
    output_dir=f"AutocompleteLM-{get_time_dir()}",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    evaluation_strategy="steps",
    do_train=True,
    do_eval=True,
    eval_steps=1_000,
    logging_steps=1_000,
    gradient_accumulation_steps=8,
    # num_train_epochs=1,
    weight_decay=.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=2_000,
    fp16=True,
    num_train_epochs=5,
    save_total_limit=1,
)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    compute_metrics=rouge,
    train_dataset=mapped['train'],
    eval_dataset=mapped['test'],
)

In [48]:
# config = AutoConfig.from_pretrained(
#     "gpt2",
#     vocab_size = len(tokenizer),
#     n_ctx = tokenizer.model_max_length,
#     bos_token_id=tokenizer.bos_token_id,
#     eos_token_id=tokenizer.eos_token_id,
# )
# model = GPT2LMHeadModel(config)
# model_size = sum(t.numel() for t in model.parameters())
# print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")
# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [49]:
trainer.train()

c:\Users\sizzf\.conda\envs\torch\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1345 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ c:\Users\sizzf\.conda\envs\torch\lib\site-packages\transformers\trainer.py:1664 in train         │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ c:\Users\sizzf\.conda\envs\torch\lib\site-packages\transformers\trainer.py:1940 in               │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1937 │   │   │   │   │   with model.no_sync():                                                 │
│   1938 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1939 │   │   │   │   else:                                                                     │
│ ❱ 1940 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1941 │   │   │   │                                                                             │
│   1942 │   │   │   │   if (                                                                      │
│   1943 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ c:\Users\sizzf\.conda\envs\torch\lib\site-packages\transformers\trainer.py:2735 in training_step │
│                                                                                                  │
│   2732 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2733 │   │                                                                                     │
│   2734 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2735 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2736 │   │                                                                                     │
│   2737 │   │   if self.args.n_gpu > 1:                                                           │
│   2738 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ c:\Users\sizzf\.conda\envs\torch\lib\site-packages\transformers\trainer.py:2767 in compute_loss  │
│                                                                                                  │
│   2764 │   │   │   labels = inputs.pop("labels")           